In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from committee_learning.initial_conditions import SymmetricInitialConditions
from committee_learning.simulation import Simulation
from committee_learning.ode import SquaredActivationODE
from committee_learning.result import SimulationResult, SquareODEResult

import numpy as np

d_list = [10000]
p = 8
k = 4
gamma0 = 1.
noise = 1e-3
dt = 1e-3
epsilon = 1e-2
q0 = 2.

def risk(q):
  return 0.5 * ((1.+2/k) - 2*q + (1+2/p)*q**2)

def qf():
  discr = (p + 2*gamma0)**2 - 4*p*(p+4)/(p+2)*(1+gamma0/p*(1+2/k+noise))
  if discr < 0.:
    print('No fixed point for q!')
  return (
    (p + 2*gamma0 - np.sqrt(discr))/(2*gamma0*(p+4)),
    (p + 2*gamma0 + np.sqrt(discr))/(2*gamma0*(p+4))
  )

qf0 = p/(p+2)
qf1, qf2 = qf()
R0 = risk(q0)
R_minimum = risk(qf0)
R_qf1 = risk(qf1)
R_qf2 = risk(qf2)

print(f'Actual initial value:\t{q0}')
print(f'Diverging threeshold:\t{qf2}')
print(f'Attractor:\t\t{qf1}')
print(f'Risk minimum q:\t\t{qf0}')

ic = SymmetricInitialConditions(p,k,epsilon,q0)

Actual initial value:	2.0
Diverging threeshold:	0.7239712567903925
Attractor:		0.10936207654294088
Risk minimum q:		0.8


### Simulations

In [3]:
simrs = {}
for d, Wt, W0 in ic.simulations(d_list):
  print(f'Simulating d = {d}')
  sim = Simulation(d,p,k,noise,Wt,gamma0,'square',W0)
  simr = SimulationResult(f'test_smallepsilon{epsilon}')
  simr.from_file_or_run(sim, path='computation-database/sim/',decades=2+np.log10(d))
  simrs[d] = simr

Simulating d = 10000


### ODE

In [4]:
ode = SquaredActivationODE(p,k,noise,gamma0,ic.P,ic.Q,ic.M,dt)
oder = SquareODEResult('', )
oder.from_file_or_run(ode, decades=2, path='computation-database/ode/')

In [8]:
import matplotlib.pyplot as plt
from committee_learning.utilities import plot_style

with plot_style():
  fig, ax = plt.subplots(figsize=(2.75,2.75))
  ax.set_xscale('log')
  ax.set_yscale('log')
  ax.set_xlabel(r'$t$')
  ax.set_ylabel(r'$\mathcal{R}$')
  # ax.set_xlim(min(ode.saved_times), max(ode.saved_times))
  # Global 
  ax.set_xlim(1e-3,1e2)
  

  # Lines
  ax.axhline(y=R0,ls='-.',lw=0.7,label=r'$\mathcal{R}{(0)}$',color='orange')
  ax.axhline(y=R_minimum,ls='-.',lw=0.7,label=r'$\varepsilon=0$ theo. min.',color='blue')
  # ax.axhline(y=R_qf1,ls='-.',lw=0.7,label=r'$\mathcal{R}{(q_f^1)}$',color='green')
  # ax.axhline(y=R_qf2,ls='-.',lw=0.7,label=r'$\mathcal{R}{(q_f^2)}$',color='red')

  # Plot simulations
  for d, s in simrs.items():
    t = np.array(s.steps) / d
    ax.plot(t, s.risks, label=f'$d={s.d}$',ls='--',lw=1.,marker='o',ms=5., c='m')
  # Plot ODE
  ax.plot(oder.times, oder.risks, label='ODE')

  ax.legend()
  fig.savefig('figures/example-small-eps.pdf', bbox_inches = 'tight')

  # # Zoom
  ax.set_xlim(1e-3,1e2)
  ax.set_ylim(0.19,1.4)
  fig.savefig('figures/example-small-eps-zoom.pdf', bbox_inches = 'tight')